**DYNAMIC PROTEIN LOCALIZATION**

In [2]:
import numpy as np
import pandas as pd

In [8]:
file_path = 'StaticData.txt'
vec = [
    "orfid", "yORF", "gene name", "GFP tagged", "GFP visualized?", "TAP visualized?", 
    "abundance", "error", "localization summary", "ambiguous", "mitochondrion", 
    "vacuole", "spindle pole", "cell periphery", "punctate composite", 
    "vacuolar membrane", "ER", "nuclear periphery", "endosome", "bud neck", 
    "microtubule", "Golgi", "late Golgi", "peroxisome", "actin", "nucleolus", 
    "cytoplasm", "ER to Golgi", "early Golgi", "lipid particle", "nucleus", "bud"
]

static_data = pd.read_csv(file_path, delimiter='\t', header=None, index_col=None, skipinitialspace=True)
static_data = static_data.iloc[:, :-2]
static_data.columns = vec
# we delete all the columns with true/false because they are included in localization summary
static_data = static_data.iloc[:, 1:9]

We start working on the STATIC DATA

In [9]:
static_data

,yORF,gene name,GFP tagged,GFP visualized?,TAP visualized?,abundance,error,localization summary
0,YAL001C,TFC3,tagged,visualized,TAP visualized,125,NaN,"cytoplasm,nucleus"
1,YAL002W,VPS8,tagged,visualized,TAP visualized,736,NaN,endosome
2,YAL005C,SSA1,tagged,visualized,TAP visualized,269000,NaN,cytoplasm
3,YAL007C,ERP2,tagged,visualized,TAP visualized,26300,NaN,ER
4,YAL008W,FUN14,tagged,visualized,TAP visualized,1920,NaN,mitochondrion
...,...,...,...,...,...,...,...,...
6229,YPR200C,ARR2,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN
6230,YPR201W,ARR3,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN
6231,YPR202W,NaN,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN
6232,YPR203W,NaN,tagged,not visualized,not TAP visualized,not visualized,NaN,NaN


In [11]:
print(np.sum(static_data["error"].isna()))
print(np.sum(static_data["localization summary"].isna()))

6028
2078


CHIEDERE:

- Quando l'errore è nan, cioè cosa rappresenta

- Cosa significano alcune colonne del dataset statico


In [12]:
# We export the table as csv

output_file_path = 'static_data.csv'
static_data.to_csv(output_file_path, index=False)